In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
df = pd.read_csv('C:\\Users\\Hp\\Desktop\\MyWorkspaces\\My Analyst Project\\Hospital Data\\data\\csv\\FY_2022-2025.csv')
df.shape

In [ ]:
df.info()
# Flag missing ERRs (common for CABG / THA-TKA)
missing = df.filter(like='ERR').isna().sum().sort_values(ascending=False)
missing.head()

In [ ]:
penalty_cols = [c for c in df.columns if 'Penalty indicator' in c]
df['Any_Penalty'] = df[penalty_cols].eq('Y').any(axis=1)

penalty_summary = (
    df[penalty_cols + ['Any_Penalty']]
    .melt(var_name='Condition', value_name='Penalty')
    .assign(Condition=lambda x: x['Condition'].str.extract(r'for (.+?)\s*$')[0])
    .query("Penalty == 'Y'")
    .groupby('Condition').size()
    .rename('Hospitals_Penalized')
    .sort_values(ascending=False)
)
penalty_summary

In [ ]:
sns.histplot(df['Payment reduction percentage'], bins=30, kde=True)
plt.title('Distribution of Medicare Payment Reductions (%)')
plt.show()

top_10_penalized = (
    df.nlargest(10, 'Payment reduction percentage')
    [['Hospital CCN', 'Payment reduction percentage', 'Peer group assignment']]
)
top_10_penalized

In [ ]:
peer_stats = (
    df.groupby('Peer group assignment')
      .agg(
          Hospitals=('Hospital CCN', 'count'),
          Avg_Payment_Reduction=('Payment reduction percentage', 'mean'),
          Any_Penalty_Rate=('Any_Penalty', 'mean'),
          Mean_DUAL=('Dual proportion', 'mean')
      )
      .round(3)
)
peer_stats

In [ ]:
# Build tidy long table of ERR & Penalty
long = []
for cond in ['AMI', 'COPD', 'HF', 'pneumonia', 'CABG', 'THA/TKA']:
    long.append(
        df[['Hospital CCN', 'Peer group assignment', f'ERR for {cond}', f'Penalty indicator for {cond}']]
        .rename(columns={
            f'ERR for {cond}': 'ERR',
            f'Penalty indicator for {cond}': 'Penalty'
        })
        .assign(Condition=cond)
    )
long = pd.concat(long)

# Ensure ERR is numeric and drop NaNs
long['ERR'] = pd.to_numeric(long['ERR'], errors='coerce')
long = long.dropna(subset=['ERR'])

# Top 10 worst ERRs with penalties
worst = (
    long.query("Penalty == 'Y'")
        .nlargest(10, 'ERR')
        .reset_index(drop=True)
)
worst[['Hospital CCN', 'Condition', 'ERR', 'Peer group assignment']]

In [ ]:
numeric = df.select_dtypes(include='number').drop(columns=['Year'])  # constant
corr = numeric.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)
plt.title('Correlation matrix of numeric features')
plt.show()

In [ ]:
# 1. High-priority hospitals (≥1.0% payment reduction + any penalty)
priority = df[df['Payment reduction percentage'] >= 1.0]

# 2. Peer groups with lowest performance
underperforming_peers = peer_stats[peer_stats['Any_Penalty_Rate'] > 0.5]

# 3. Conditions driving the most penalties
conditions_to_watch = penalty_summary.head(3).index.tolist()

print("ACTIONABLE INSIGHTS")
print("-" * 40)
print(f"• {len(priority)} hospitals face ≥1% payment reduction.")
print(f"• Peer groups with >50% penalty rate: {underperforming_peers.index.tolist()}")
print(f"• Top 3 conditions to target for improvement: {conditions_to_watch}")

1️⃣ ## Which hospitals lose the most Medicare dollars in absolute terms?

In [ ]:
# AMI-specific columns
drg_col = 'DRG payment ratio for AMI'
vol_col = 'Number of eligible discharges for AMI'

# Ensure both are numeric
df[drg_col] = pd.to_numeric(df[drg_col], errors='coerce')
df[vol_col] = pd.to_numeric(df[vol_col], errors='coerce')

# Dollar loss estimate
df['AMI_DollarLoss'] = (1 - df[drg_col]).fillna(0) * df[vol_col].fillna(0)

# Top 10 hospitals by estimated loss
top_dollar_loss = (
    df[['Hospital CCN', 'AMI_DollarLoss']]
    .query("AMI_DollarLoss > 0")
    .sort_values('AMI_DollarLoss', ascending=False)
    .head(10)
)
top_dollar_loss

Peer-group median ERR vs. your hospital’s ERR – instant benchmarking

In [ ]:
def benchmark_condition(condition='AMI'):
    base = df[['Hospital CCN', 'Peer group assignment', f'ERR for {condition}']].dropna()

    base[f'ERR for {condition}'] = pd.to_numeric(base[f'ERR for {condition}'], errors='coerce')

    medians = base.groupby('Peer group assignment')[f'ERR for {condition}'].median()

    base = base.merge(medians.rename(f'Median_ERR_{condition}'), on='Peer group assignment')

    base['Above_Peer_Median'] = base[f'ERR for {condition}'] > base[f'Median_ERR_{condition}']
    
    return base[['Hospital CCN', f'ERR for {condition}', f'Median_ERR_{condition}', 'Above_Peer_Median']]

benchmark_condition('AMI').head()

In [ ]:
sns.boxplot(x='Any_Penalty', y='Dual proportion', data=df)
plt.title('Dual-eligible share by penalty status')
plt.show()


4️⃣ ##### Smallest hospitals (Peer 1) that still escape penalties – best-practice candidates

In [ ]:
mask = (df['Peer group assignment'] == 1) & (~df['Any_Penalty'])
good_small = (
    df.loc[mask, ['Hospital CCN', 'Dual proportion']]
      .nsmallest(10, 'Dual proportion')
)
good_small

In [ ]:
vol_err = (
    df[['Number of eligible discharges for AMI', 'ERR for AMI']]
    .dropna()
    .assign(**{'ERR for AMI': lambda x: pd.to_numeric(x['ERR for AMI'], errors='coerce')})
)
sns.regplot(data=vol_err, x='Number of eligible discharges for AMI', y='ERR for AMI')
plt.title('AMI Volume vs ERR')
plt.show()

In [ ]:
def worst_metric_per_hospital():
    cols = [c for c in df.columns if 'ERR' in c and 'Penalty' not in c]
    tidy = (df[['Hospital CCN'] + cols]
            .melt(id_vars='Hospital CCN', var_name='Metric', value_name='ERR')
            .dropna())
    tidy['ERR'] = pd.to_numeric(tidy['ERR'], errors='coerce')
    # Highest ERR per hospital
    return (tidy.sort_values('ERR', ascending=False)
                .drop_duplicates('Hospital CCN')
                .head(15))
worst_metric_per_hospital()

In [ ]:
penalty_heatmap = (
    long.assign(Penalty=lambda x: x['Penalty'] == 'Y')
        .groupby(['Condition', 'Peer group assignment'])['Penalty']
        .mean()
        .unstack(fill_value=0)
)
sns.heatmap(penalty_heatmap, annot=True, fmt='.1%', cmap='Reds')
plt.title('Penalty rate by Condition & Peer Group')
plt.show()

In [ ]:
def z_score_condition(condition='HF'):
    sub = df[['Hospital CCN', 'Peer group assignment', f'ERR for {condition}']].dropna()
    sub[f'ERR for {condition}'] = pd.to_numeric(sub[f'ERR for {condition}'], errors='coerce')
    sub['z_ERR'] = (sub.groupby('Peer group assignment')[f'ERR for {condition}']
                      .transform(lambda x: (x - x.mean()) / x.std()))
    return sub.sort_values('z_ERR', ascending=False).head(10)
z_score_condition('HF')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

features = ['ERR for AMI', 'ERR for HF', 'ERR for COPD', 'Dual proportion']

# Build clean numeric matrix
Xy = df[features + ['Any_Penalty']].copy()
Xy[features] = Xy[features].apply(pd.to_numeric, errors='coerce')
Xy = Xy.dropna()                       # <- remove any NaN rows
X = Xy[features]
y = Xy['Any_Penalty'].astype(int)

# Model
clf = LogisticRegression(max_iter=1000).fit(StandardScaler().fit_transform(X), y)

# Coefficients
coef = pd.Series(clf.coef_[0], index=features).sort_values(key=abs, ascending=False)
coef